In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

##############################################
# You must change the seed to
#  your student ID(last three number).
##############################################
torch.manual_seed(0)
np.random.seed(0)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent','Mozilla/5.0')]
urllib.request.install_opener(opener)

# Data Loader


In [ ]:
import os
!gdown --id '1fOH2ahsT4-EHwMa7ftcoxSpad2X3EJxa' --output MNIST.zip
if not os.path.exists('/content/drive/MyDrive/Colab Notebooks/data/MNIST/MNIST'):
 os.makedirs('/content/drive/MyDrive/Colab Notebooks/data/MNIST/MNIST')
!unzip MNIST.zip -d '/content/drive/MyDrive/Colab Notebooks/data/MNIST/MNIST'


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     ])

trainset = torchvision.datasets.MNIST(root='/content/drive/MyDrive/Colab Notebooks/data/MNIST', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='/content/drive/MyDrive/Colab Notebooks/data/MNIST', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')

# Visualize Images

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(32)))

# Build Model Example

In [ ]:
## This is an simple example of model building by init parameters and adding layer.

class Net_S(nn.Module):
    def __init__(self, n , x):
        super(Net_S, self).__init__()
        # Initialize object variables
        self.layer = n
        self.neurons = x

        #convolutional layer
        self.conv = torch.nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.MaxPool2d(2,2),
            nn.ReLU(),
            nn.Conv2d(16,64,5),
            nn.MaxPool2d(2,2),
            nn.ReLU(),
        )

        # build sequential model
        self.hid = torch.nn.Sequential()
        self.hid.add_module('Lin'+str(1), torch.nn.Linear(64*4*4, 128)) # appends a Linear layer to hidden 
        self.hid.add_module("Lin2", torch.nn.Linear(128, 64)) 
        self.hid.add_module("Lin3", torch.nn.Linear(64 , 10))

        # call class function
        self._print_layers()

    def _print_layers(self):
        print("number of layer: {}  number of neurons: {}".format(self.layer, self.neurons))

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 64*4*4)
        x = self.hid(x)

        return x




In [ ]:
net3 =  Net_S(5, 3)
print(net3)

# Hyper parameters settings

In [ ]:
##############################################
# Remember change net to your 'class Net name'!
##############################################
net =  Net_S(3,4)

## Use GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
net = net.to(device)

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
epochs = 20


# net.eval()
net.train()


# Training routine

In [ ]:
# Training model
for epoch in tqdm(range(epochs)):# loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # print(inputs.shape)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches if epochs equals 20
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 999))
            running_loss = 0.0

print('Finished Training')


# Testing model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))